In [5]:
# Re-create the game engine 
test_case_1 = {'m': 1,q  'l0': '..............................', 'v': 1, }
# Create a simulation
test_case_1 = {'m': 1, 'l0': '...0...0...', 'v': 1, }


initial_state = { 'm': 1, 'v': 1, 'turn': 0, 'x': 0, 'is_jumping': False, 'speed': 0, 
                 'is_alive': True,  "lane": '....0.', 
                 "moto_lane" : None, "survived": False,'end_of_lane': False
                }


# TODO - Create a pydantic typed state object
def update_state(state, action):
    state['turn'] += 1
    
    updated_x = state['x'] + state['speed']
    
    if updated_x > len(state['lane']):
        interval = state['lane'][state['x']:]
        state['end_of_lane'] = True 
    else:
        interval  = state['lane'][state['x']:updated_x + 1]

    if not state['is_jumping'] and '0' in interval:
        state['is_alive'] = False
        print("Motorcycle DIED")

    if state['is_alive'] and state['end_of_lane']:
        print("Motorcycle MADE IT")

    # Update the x position
    state['x'] += state['speed']

    
    state['moto_lane'] = state['lane'][:state['x']] + '🏍️' + state['lane'][state['x']+1:]
    
    state['is_jumping'] = False
    # Take Action 
    match action:
        case 'JUMP':
            state['is_jumping'] = True
        case 'SPEED':
            state['speed'] += 1
        case 'SLOW':
            if state['speed'] > 0:
                state['speed'] -= 1
        case 'WAIT':
            pass

    return state


In [419]:
# Progression 1: Build a working algorithm for a Single Lane, Single Bike 

initial_state = { 'turn': 0, 'x': 0, 'is_jumping': False, 'speed': 3, 
                 'is_alive': True, "moto_lane" : None, 'end_of_lane': False, 'next_action': None
                }

def findValidSequences(initial_state, lane):
    
    # Adding additional empty spaces for a case where the last entry is 0
    # Per comment & implementation 
    modified_lane = lane + '.....'
    
    def update_state(state, action):  
        match action:
            case 'JUMP':
                state['is_jumping'] = True
            case 'SPEED':
                state['speed'] += 1
            case 'SLOW':
                if state['speed'] > 0:
                    state['speed'] -= 1
            case 'WAIT':
                pass

        start = state['x']
        end =  state['x'] + state['speed'] 
        
        # End of lane. TODO: Handle edge case where the last entry is a 0. 
        if end >= len(modified_lane):
            state['end_of_lane'] = True
            interval = modified_lane[start:]
        else:
            interval = modified_lane[state['x']:end + 1]
 
        # Case 1 - Motorcycle drove over a hole or # Case 2 - Landed on a hole
        if (not state['is_jumping'] and '0' in interval) or (interval[-1] == '0'):
            state['is_alive'] = False
          
        # Update state based on action
        state['x'] = end
        state['is_jumping'] = False
        state['turn'] += 1
        # Visual display for debugging
        state['moto_lane'] = lane[:state['x']] + '🏍️' + modified_lane[state['x']+1:]
        return state

    def is_valid_end_state(state):
        return state['is_alive'] and state['end_of_lane']
    
    def is_dead_state(state):
        return not state['is_alive'] or state['turn'] > 6
    
    # Optimization - returns a list of valid actions given the current state
    # TODO: Later 
    # TODO: Experiment with removing "WAIT"
    # TODO: Enum
    def valid_actions(state):
        return ['SPEED', 'JUMP', 'SLOW', 'WAIT']
                
    valid_sequences = []

    def backtracking(state, sequence_of_actions, num_results=1):
        
        if (len(valid_sequences) >= num_results):
            return
        
        if is_valid_end_state(state):
            valid_sequences.append(list(sequence_of_actions))
            return 
    
        if is_dead_state(state):
            return 
        
        for action in valid_actions(state):
            sequence_of_actions.append(action)
            print(sequence_of_actions)
            # Pass in a deep copy and get back a reference to a mutated state
            updated_state = update_state(state.copy(), action)
            backtracking(updated_state, sequence_of_actions)
            sequence_of_actions.pop()

    backtracking(initial_state, [])
        
    return valid_sequences


In [420]:
findValidSequences(initial_state, '.0..0.......0...0')

['SPEED']
['JUMP']
['JUMP', 'SPEED']
['JUMP', 'JUMP']
['JUMP', 'JUMP', 'SPEED']
['JUMP', 'JUMP', 'SPEED', 'SPEED']
['JUMP', 'JUMP', 'SPEED', 'JUMP']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'SPEED']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP', 'SPEED']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP', 'JUMP']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP', 'SLOW']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP', 'WAIT']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'SLOW']
['JUMP', 'JUMP', 'SPEED', 'JUMP', 'WAIT']
['JUMP', 'JUMP', 'SPEED', 'SLOW']
['JUMP', 'JUMP', 'SPEED', 'WAIT']
['JUMP', 'JUMP', 'JUMP']
['JUMP', 'JUMP', 'SLOW']
['JUMP', 'JUMP', 'WAIT']
['JUMP', 'SLOW']
['JUMP', 'WAIT']
['SLOW']
['WAIT']


[['JUMP', 'JUMP', 'SPEED', 'JUMP', 'JUMP', 'SPEED']]

In [387]:
# Multiple Lanes - change from dictionary to class based approach
class State():
    """
    M: Number of bikes
    V: Minimum number of bikes to survive
    l0, l1, l2, l3: Lanes of the roads
    s: The initial starting speed from the game
    """
    def __init__(self, M,V,l0,l1,l2,l3):
        self.x = 0
        self.turn = 0
        self.M = M
        self.V = V
        self.road = [l0,l1,l2,l3]
        self.road_length = len(l0)
        self.bike_data = {} # Grab from first iteration of the game
        self.speed = 0 # Grab from first iteration of the game
        self.is_jumping = False
        self.top_lane_occupied = False
        self.is_changing_up = False
        self.is_changing_down = False
        self.num_alive = M # Assume all bikes are alive in the initial state and >= v
        self.end_of_road = False
    
    def __str__(self):
        return f'Speed: {self.speed} X_Position: {self.x} Bike Data:{self.bike_data} Num Alive:{self.num_alive} End: {self.end_of_road}'
    
    def set_bike_data(self, i, y, a):
        self.bike_data[i] = {'Y': y, 'A': a}

    def set_speed(self, s):
        self.speed = s

    
    def update_state_on_action(self, action):
        match action:
            case 'JUMP':
                self.is_jumping = True
            case 'SPEED':
                self.speed += 1
            case 'SLOW':
                if self.speed > 0:
                    self.speed -= 1
            case 'UP':
                if not self.is_top_lane_occupied():
                    self.is_changing_up = True
                    for data in self.bike_data.values():
                        # Decrement y to Move UP
                        data['Y'] -= 1
            case 'DOWN':
                if not self.is_bottom_lane_occupied():
                    self.is_changing_down = True
                    for data in self.bike_data.values():
                        # Increment y to Move DOWN
                        data['Y'] += 1
            case 'WAIT':
                pass
            
        self.move_and_check_alive()
    
    
    def move_and_check_alive(self):

        for bike in self.bike_data.values():
            # Cases for hitting a pothole            
            # Assume all equal lenght
            x_interval = self.x + self.speed
        
    
            if x_interval > self.road_length: 
                self.end_of_road = True 
                x_interval = self.road_length - 2 # Think about this - might need to add more ....
            
            # Case 1: Changing up              
            if self.is_changing_up and (('0' in self.road[bike['Y'] + 1][self.x:x_interval]) or ('0' in self.road[bike['Y']][self.x:x_interval+ 1])):
                bike['A'] = False
                self.num_alive -= 1
            # Case 2: Changing down
            elif self.is_changing_down and (('0' in self.road[bike['Y'] - 1][self.x:x_interval]) or ('0' in self.road[bike['Y']][self.x:x_interval + 1])):
                bike['A'] = False
                self.num_alive -= 1
            # Case 3: Jump onto a pothole
            elif (self.is_jumping and self.road[bike['Y']][x_interval] == '0'):
                bike['A'] = False
                self.num_alive -= 1
            # Case 4: Drive over a pot hole (no lane change)
            elif ('0' in self.road[bike['Y']][self.x:(x_interval) + 1]) and not self.is_jumping:
                bike['A'] = False
                self.num_alive -= 1

        # Reset the action sepecific state varables
        self.is_changing_down = False 
        self.is_changing_up = False
        self.is_jumping = False 
        self.x += self.speed
        self.turn += 1
    

    def is_top_lane_occupied(self):
        for data in self.bike_data.values():
            if data['Y'] == 0:
                print('Unable to MOVE UP')
                return True
    
    def is_bottom_lane_occupied(self):
        for data in self.bike_data.values():
            if data['Y'] == 3:
                print('Unable to MOVE DOWN')
                return True
        
        return False
    


In [409]:
import copy
l0, l1, l2, l3 = '..............................', '..............................', '...........0..................','..............................'
m,v,s = 1, 1, 3
bike_data = [[2, True]]
game = State(m,v,l0,l1,l2,l3)
for i, data in enumerate(bike_data):
    game.set_bike_data(i, data[0], data[1])

game.set_speed(s)
print(game)



In [410]:
res = findValidSequence()    

Speed: 3 X_Position: 0 Bike Data:{0: {'Y': 2, 'A': True}} Num Alive:1 End: False
['SPEED', 'SPEED', 'SPEED'] Speed: 6 X_Position: 15 Bike Data:{0: {'Y': 2, 'A': False}} Num Alive:0 End: False
['SPEED', 'SPEED', 'UP'] Speed: 6 X_Position: 21 Bike Data:{0: {'Y': 1, 'A': False}} Num Alive:0 End: False
['SPEED', 'SPEED', 'DOWN'] Speed: 6 X_Position: 27 Bike Data:{0: {'Y': 2, 'A': False}} Num Alive:0 End: False
['SPEED', 'SPEED', 'JUMP'] Speed: 6 X_Position: 33 Bike Data:{0: {'Y': 2, 'A': False}} Num Alive:0 End: True
['SPEED', 'SPEED', 'SLOW'] Speed: 5 X_Position: 38 Bike Data:{0: {'Y': 2, 'A': False}} Num Alive:0 End: True
['SPEED', 'SPEED', 'WAIT'] Speed: 5 X_Position: 43 Bike Data:{0: {'Y': 2, 'A': False}} Num Alive:0 End: True
['SPEED', 'SPEED'] Speed: 5 X_Position: 9 Bike Data:{0: {'Y': 2, 'A': True}} Num Alive:1 End: False
['SPEED', 'UP'] Speed: 5 X_Position: 14 Bike Data:{0: {'Y': 1, 'A': False}} Num Alive:0 End: False
['SPEED', 'DOWN'] Speed: 5 X_Position: 19 Bike Data:{0: {'Y': 2,

In [416]:
l0 = l1 = l2 = l3 = '..........000......0000..............000000.............'
m,v,s = 4,4,2
game = State(m,v,l0,l1,l2,l3)
game.bike_data = {0: {'Y': 0, 'A': 1}, 1: {'Y': 1, 'A': 1}, 2: {'Y': 2, 'A': 1}, 3: {'Y': 3, 'A': 1}}

def findValidSequence(num_results = 1, game):
    # Optimization - Try reduce state space later
    def valid_actions():
        return ['SPEED', 'UP', 'DOWN', 'JUMP', 'SLOW', 'WAIT']
    
    valid_sequences = [] 

    def backtracking(sequence_of_actions, game):
        
        if (len(valid_sequences) >= num_results):
            return
    
        
        # Successful end state - backtrack
        if game.num_alive >= game.V and game.end_of_road:
            valid_sequences.append(list(sequence_of_actions))
            return 
    
        # Dead state - backtrack
        if game.num_alive < game.V or game.turn > 15:
            return
        
        for action in valid_actions():
            game = copy.deepcopy(game)
            sequence_of_actions.append(action)
            game.update_state_on_action(action)
            backtracking(sequence_of_actions, game)
            print(sequence_of_actions, game)
            sequence_of_actions.pop()

    backtracking([], game) 
    
    return valid_sequences
    

SyntaxError: non-default argument follows default argument (2764272332.py, line 6)

In [418]:
findValidSequence(1, game)

TypeError: findValidSequence() takes from 0 to 1 positional arguments but 2 were given

In [426]:
import heapq


priority_queue = []

heapq.heappush(priority_queue, (4, 'a','d'))
heapq.heappush(priority_queue, (2, 'b'))
heapq.heappush(priority_queue, (3, 'c'))

while priority_queue:
    print(heapq.heappop(priority_queue))


(2, 'b')
(3, 'c')
(4, 'a', 'd')
